In [1]:
import aiofiles
import jinja2
from colabcode import ColabCode
import pefile
from fastapi import FastAPI
import multipart
from sklearn.pipeline import Pipeline
import pandas as pd
import os
import math
import numpy as np
from io import BytesIO
from keras.applications import imagenet_utils
import tensorflow as tf 
import itertools
import pefile
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import uvicorn
from fastapi import FastAPI, File, UploadFile
from starlette.responses import RedirectResponse
import cv2
import tempfile
import uvicorn
from tensorflow.keras.utils import img_to_array
from fastapi.templating import Jinja2Templates
from PIL import Image

In [2]:
cc = ColabCode(port=12000, code=False)
import joblib
import itertools

In [3]:
def createDataframeFromPEdump(pe):

	dosHeaders = ['e_magic', 'e_cblp', 'e_cp', 'e_crlc', 'e_cparhdr',
	       'e_minalloc', 'e_maxalloc', 'e_ss', 'e_sp', 'e_csum', 'e_ip', 'e_cs',
	       'e_lfarlc', 'e_ovno', 'e_oemid', 'e_oeminfo', 'e_lfanew']
	fileHeaders=['Machine',
	       'NumberOfSections', 'PointerToSymbolTable',
	       'NumberOfSymbols', 'SizeOfOptionalHeader', 'Characteristics']
	optionalHeaders=['Magic',
	       'MajorLinkerVersion', 'MinorLinkerVersion', 'SizeOfCode',
	       'SizeOfInitializedData', 'SizeOfUninitializedData','AddressOfEntryPoint', 'BaseOfCode',
	        'ImageBase', 'SectionAlignment','FileAlignment', 'MajorOperatingSystemVersion',
	       'MinorOperatingSystemVersion', 'MajorImageVersion', 'MinorImageVersion',
	       'MajorSubsystemVersion', 'MinorSubsystemVersion', 'SizeOfHeaders',
	       'CheckSum', 'SizeOfImage', 'Subsystem', 'DllCharacteristics',
	       'SizeOfStackReserve', 'SizeOfStackCommit', 'SizeOfHeapReserve',
	       'SizeOfHeapCommit', 'LoaderFlags', 'NumberOfRvaAndSizes']
	imageDirectory = ['ImageDirectoryEntryExport', 'ImageDirectoryEntryImport',
	       'ImageDirectoryEntryResource', 'ImageDirectoryEntryException',
	       'ImageDirectoryEntrySecurity']

	dheaders = {}
	fheaders = {}
	oheaders={}
	imd1 = {}

	for x in dosHeaders:
		dheaders[x] = getattr(pe.DOS_HEADER,x)
	df = pd.DataFrame(dheaders,index=[0])

	for i in fileHeaders:
		fheaders[i] = getattr(pe.FILE_HEADER,i)
	df = pd.concat([df,(pd.DataFrame(fheaders,index=[0]))],axis=1)

	for y in optionalHeaders:
		oheaders[y] = getattr(pe.OPTIONAL_HEADER,y)
	df = pd.concat([df,(pd.DataFrame(oheaders,index=[0]))],axis=1)

	for q in pe.OPTIONAL_HEADER.DATA_DIRECTORY:
	    imd1[q.name] = q.VirtualAddress
	imd1 = dict(itertools.islice(imd1.items(),5))
	df = pd.concat([df,(pd.DataFrame(imd1,index=[0]))],axis=1)

	return df

def getPredictions(df):
    load_scaler = joblib.load(open(r'../components/stackingscaler.pkl','rb'))
    load_skpca = joblib.load(open(r'../components/stackingpca.pkl','rb'))
    load_model = joblib.load(open(r'../components/stackingrf_model.pkl','rb'))
    pipe = Pipeline([('scale', load_scaler),('pca', load_skpca),('rf_model', load_model)])
    df = np.array(df)
    df = df.reshape(1,-1)
    results = pipe.predict_proba(df)
    pred = pipe.predict(df)
    return (results[0],pred[0])

In [4]:
#creating fastApi app 
from fileinput import filename


app_desc = """<h2> Try uploading a Portable Executable(PE) file"""
app = FastAPI(description = app_desc)
templates = Jinja2Templates(directory="html")
@app.get("/",include_in_schema=False)
async def index():
	return RedirectResponse(url="/docs")

@app.post("/predict")
def parse(file: UploadFile = File(...)):
    extension = os.path.splitext(file.filename)[1]
    _, path = tempfile.mkstemp(prefix='parser_', suffix=extension)

    with open(path, 'ab') as f:
        for chunk in iter(lambda: file.file.read(10000), b''):
            f.write(chunk)

    # extract content
    content = pefile.PE(path,fast_load=True)
    dataframe = createDataframeFromPEdump(content)
    binary_preds = getPredictions(dataframe)
    if binary_preds[1] == 1:
      return {'Name file' : file.filename, 'response':'Malicious file','predictions':binary_preds[0][1]*100}
    else:
      return {'Name file' : file.filename, 'Response': 'Not Malicious file.','Malicious percentage':binary_preds[0][1]*100}
    #remove temp file
    os.close(_)
    os.remove(path)